<a href="https://colab.research.google.com/github/soroush-04/Apple_Stock_Prediction/blob/main/AppleStockPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas_datareader

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
pip install --upgrade pandas


In [ ]:
pip install --upgrade pandas-datareader

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as data

In [ ]:
import yfinance as yf

start = '2010-01-01'
end = '2019-12-31'

df = yf.download('AAPL', start=start, end=end)


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df = df.reset_index()
df.head()

In [ ]:
df = df.drop(['Date'], axis = 1)

In [ ]:
df

In [ ]:
df = df.drop(['Adj Close'], axis = 1)

In [ ]:
df

In [ ]:
plt.plot(df.Close)
# Here x axis is sample number and Y axis is closing prices

In [ ]:
ma100 = df.Close.rolling(100).mean()
ma100
# moving average = from first 100 days it will calculate last 100 days moving average

In [ ]:
# ploting the Average value
plt.figure(figsize = (12,6))
plt.plot(df.Close, 'g')
plt.plot(ma100, 'b')

In [ ]:
#calcuulating average og 200 days
ma200 = df.Close.rolling(200).mean()
ma200

In [ ]:
# ploting the Moving average of 100, 200 days with original close price Average value
plt.figure(figsize = (12,6))
plt.plot(df.Close, 'r')
plt.plot(ma100, 'b')
plt.plot(ma200, 'g')

In [ ]:
df.shape

In [ ]:
# splitting Data into Training and Testing
data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.7)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.7): int(len(df))])

In [ ]:
print(data_training.shape)
print(data_testing.shape)

In [ ]:
data_training.head()

In [ ]:
data_testing.head()

In [ ]:
# Scaling down the data between 0 and 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))


In [ ]:
#scaler.fit automaticcaly gives an array
data_training_array = scaler.fit_transform(data_training)
data_training_array

In [ ]:
data_training_array.shape

In [ ]:
x_train = []
y_train = []

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i-100: i])
    y_train.append(data_training_array[i, 0])

In [ ]:
y_train


In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train

In [ ]:
# ML MODEl
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(units = 50, activation = 'relu', return_sequences = True,
              input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))



#2nd
model.add(LSTM(units = 50, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3))


#3rd
model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.4))


#4th
model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5))


model.add(Dense(units = 1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs =50 )

In [ ]:
#let save the model
model.save('keras_model.h5')

In [ ]:
data_testing.head()

In [ ]:
data_training.tail(100)

In [ ]:
past_100_days = data_training.tail(100)

In [ ]:
# final_df = past_100_days.append(data_testing, ignore_index=True)

final_df = pd.concat([past_100_days, data_testing], ignore_index=True)


In [ ]:
final_df.head()

In [ ]:
input_data = scaler.fit_transform(final_df)

In [ ]:
input_data

In [ ]:
input_data.shape

In [ ]:
x_test = []
y_test = []

for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i-100: i])
    y_test.append(input_data[i, 0])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)
print(x_test.shape)
print(y_test.shape)

In [ ]:
#making predictions

y_predicted = model.predict(x_test)

In [ ]:
y_predicted.shape

In [ ]:
y_test

In [ ]:
y_predicted

In [ ]:
scaler.scale_

In [ ]:
scale_factor = 1/0.02099517
y_predicted = y_predicted * scale_factor
y_test = y_test * scale_factor

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(y_test, 'b', label = 'Original Price')
plt.plot(y_predicted, 'r', label = 'Predicted price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()